In [1]:
import torch
import sys
sys.path.append('..')
from src.module2graph import GraphInterperterWithGamma
from src.resnet18 import ResNet18
import numpy as np

import graphviz
import itertools
import copy
from torchvision import transforms
import torchvision

import networkx as nx
from tqdm.auto import tqdm
from typing import Tuple, Dict # actually we don't need it for py>=3.9, but I have 3.8 on my laptop
from src.utils import train_loop, test_loop
#from numba import njit
from src.cifar_data import get_dataloaders


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
# forward for target model with gamma values for each edge.
# means - mean values for arguments
def forward_with_gammas(model, gammas: Dict[Tuple[str, str], torch.Tensor], 
                        means: Dict[str, torch.Tensor] = None, *torch_model_args):
    args_iter = iter(torch_model_args)
    env : Dict[str, Node] = {}
    used_edges = set()
    def load_arg(a):    
        return torch.fx.graph.map_arg(a, lambda n: env[n.name])

    def fetch_attr(target : str):
        target_atoms = target.split('.')
        attr_itr = model.graph
        for i, atom in enumerate(target_atoms):
            if not hasattr(attr_itr, atom):
                raise RuntimeError(f"Node referenced nonexistant target {'.'.join(target_atoms[:i])}")
            attr_itr = getattr(attr_itr, atom)
        return attr_itr
    named_modules = dict(model.named_modules())
    for node in model.graph.nodes:
        edges = []

        if node.op in ['call_module', 'call_function', 'output']:    
            if node.op == 'output':
                edges = [(node.args[0][0].name, node.name)]
            else:
    
                for arg in node.args:
                    if type(arg) == torch.fx.Node:  # ignore constants
                        edges.append((arg.name, node.name))
                    else:
                        edges.append(None)
            gammas_node = [(gammas[e])  if (e is not None) else 1 for e in edges ]
                
            #print (edges, gammas_node)
        if node.op == 'placeholder':
            result = next(args_iter) 
        elif node.op == 'get_attr':
            result = fetch_attr(node.target)
        elif node.op == 'call_function':
            args = [a*g + (1.0 - g) * means[str(a0)] if str(a0) in means else a*g  for a0,a,g in zip(node.args,
                                                                           load_arg(node.args), gammas_node)]
            #print (len(args), len(node.args))
            #print (node, [a for a in node.args])
            #print (node, [a.shape for a in args])
            result = node.target(*args, **load_arg(node.kwargs)) 
        elif node.op == 'call_method':
            self_obj, *args = load_arg(node.args) 
            kwargs = load_arg(node.kwargs)
            args =  [a*g + (1.0 - g) * means[str(a0)] if str(a0) in means else a*g   for a0, a,g in zip(node.args[1:], 
                                                                        args, gammas_node)]
            result = getattr(self_obj, node.target)(*args, **kwargs)
        elif node.op == 'call_module':
            args = [a*g + (1.0 - g) * means[str(a0)] if str(a0) in means else a*g   for a0, a,g in zip(node.args, 
                                                                           load_arg(node.args), gammas_node)]
            
            result = named_modules[node.target](*args, **load_arg(node.kwargs)) 
        
        result = result
        for e in edges:
            used_edges.add(e)
        
        if node.op == 'output':
            
            return result, env # currently ignorign means for output
        #print (node.args, node.name, node.op, abs(result).sum().item())
        env[node.name] = result
        
    return result

# a wrapper that takes model and uses forward_with_Gammas
class PrunedModel(torch.nn.Module):
    def __init__(self, base, prune_dict, means = None):
        super().__init__()
        self.base = base
        self.prune_dict = prune_dict
        self.means = means 
    def forward(self, x):
        return forward_with_gammas(self.base, self.prune_dict,  self.means, x)

    

In [3]:
# gets intermediate representations of nodes
def get_inter(model, *torch_model_args) -> dict:
    args_iter = iter(torch_model_args)
    env : Dict[str, Node] = {}
    used_edges = set()
    inter = {}
    def load_arg(a):    
        return torch.fx.graph.map_arg(a, lambda n: env[n.name])

    def fetch_attr(target : str):
        target_atoms = target.split('.')
        attr_itr = model.graph
        for i, atom in enumerate(target_atoms):
            if not hasattr(attr_itr, atom):
                raise RuntimeError(f"Node referenced nonexistant target {'.'.join(target_atoms[:i])}")
            attr_itr = getattr(attr_itr, atom)
        return attr_itr
    named_modules = dict(model.named_modules())
    for node in model.graph.nodes:
        edges = []

        if node.op in ['call_module', 'call_function', 'output']:    
            if node.op == 'output':
                edges = [(node.args[0][0].name, node.name)]
            else:
    
                for arg in node.args:
                    if type(arg) == torch.fx.Node:  # ignore constants
                        edges.append((arg.name, node.name))
                    else:
                        edges.append(None)
                
            #print (edges, gammas_node)
        if node.op == 'placeholder':
            result = next(args_iter) 
        elif node.op == 'get_attr':
            result = fetch_attr(node.target)
        elif node.op == 'call_function':
            
            args = load_arg(node.args)
            for a_, a in zip(node.args, args):
                inter[a_] = a
            #print (len(args), len(node.args))
            #print ([a.shape for a in load_arg(node.args)], [a.shape for a in args])
            result = node.target(*args, **load_arg(node.kwargs)) 
        elif node.op == 'call_method':
            self_obj, *args = load_arg(node.args) 
            
            for a_, a in zip(node.args[1:], args):
                inter[a_] = a
            kwargs = load_arg(node.kwargs)
            result = getattr(self_obj, node.target)(*args, **kwargs)
        elif node.op == 'call_module':
            args = load_arg(node.args)
            for a_, a in zip(node.args, args):
                inter[a_] = a
            result = named_modules[node.target](*args, **load_arg(node.kwargs)) 
        
        
        result = result
        for e in edges:
            used_edges.add(e)
    
        if node.op == 'output':
            
            return inter 
        #print (node.args, node.name, node.op, abs(result).sum().item())
        env[node.name] = result
        
    return inter


In [4]:


train_dl, test_dl = get_dataloaders([0,1,2,3,4,5,6,7], )


Files already downloaded and verified
Files already downloaded and verified


In [5]:
def module_to_graph(m: torch.nn.Module):
    graph = torch.fx.symbolic_trace(m).graph
    named_dict = dict(m.named_modules())
    edges = [] # (from, to)
    weights = {'x': 0} # node: params
    for node in graph.nodes:
        # no placeholder and call_mathod
        if node.op == 'call_module':
            n_params = sum([p.numel() for p in named_dict[node.target].parameters()])
            weights[node.name] = n_params
            assert len(node.args) == 1
            for arg in node.args:
                if type(arg) == torch.fx.Node:  # ignore constants
                    edges.append((arg.name, node.name))
        elif node.op == 'call_function':
            for arg in node.args:
                if type(arg) == torch.fx.Node:  # ignore constants
                    edges.append((arg.name, node.name))
            weights[node.name] = 0
        elif node.op == 'output':
            try:
                edges.append((node.args[0][0].name, node.name))
            except:
                edges.append((node.args[0].name, node.name))
            weights['output'] = 0
            
    return edges, {'_'.join(k.split('.')): v for k, v in weights.items()}

#forward_with_gammas(torch.fx.symbolic_trace(ResNet18()), {k: 1.0 for k in edges}, torch.randn(64, 3, 32, 32))

In [1]:
edges, weights, a, b = module_to_graph(ResNet18())
# edges, weights
edges[:10], list(weights.items())[:10]

NameError: name 'module_to_graph' is not defined

In [7]:
# getting subset to evaulate mean
edges, weights = module_to_graph(ResNet18())
# edges, weights
edges[:10], list(weights.items())[:10]



train_dl_limit, _ = get_dataloaders([0,1,2,3,4,5,6,7], train_limit=1024)# 256
len(train_dl_limit)

Using cache found in /home/legin/.cache/torch/hub/pytorch_vision_v0.10.0


Files already downloaded and verified
Files already downloaded and verified


64

In [103]:
inter = {}
inter_std = {}
model = ResNet18(8)
# модель лежит тут 
# https://github.com/bahleg/fast_nas_adapt/blob/main/cv_experiment/results/cifar_8_pretrain/model_last.ckpt
model.load_state_dict(torch.load('../model_last.ckpt', map_location='cpu'))
tr = torch.fx.symbolic_trace(model)
elem_count = 0
for x,_ in train_dl_limit:
    elem_count += x.shape[0]
    i_ = get_inter(tr, x)
    for k in i_:
        try:
            if str(k) not in inter:
                inter[str(k)] = i_[k].sum(0).detach()
                inter_std[str(k)] = [i_[k].detach()]
            else:
                inter[str(k)] += i_[k].sum(0).detach()
                inter_std[str(k)].append(i_[k].detach())
        except:
            print ('bad inter', k) # промежуточное значение для константы
        
            #inter[str(k)] = 0.0
for k in inter:
    inter[k] /= elem_count
for k in inter_std:
    inter_std[k] = torch.cat(inter_std[k], dim=0).std(0)
        

Using cache found in /home/legin/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1


In [107]:
# getting grads
gammas = [torch.nn.Parameter(torch.tensor(1.0)) for k in edges]
grads = [torch.tensor(0.0) for k in edges]
model = ResNet18(8)
model.load_state_dict(torch.load('../model_last.ckpt', map_location='cpu'))

wrapped = torch.fx.symbolic_trace(model)
crit = torch.nn.CrossEntropyLoss()
pruned = PrunedModel(wrapped, {k:g for k,g in zip(edges, gammas)}, inter)

for x,y in train_dl_limit:
    loss = crit(pruned(x)[0], y)
    gr = torch.autograd.grad(loss, gammas, allow_unused=True)
    for i in range(len(gr)):
        if gr[i] is not None and gr[i] == gr[i]:
            grads[i] += gr[i]
    

Using cache found in /home/legin/.cache/torch/hub/pytorch_vision_v0.10.0


(tensor(581.3264),
 {('x', 'model_conv1'): tensor(0.7817),
  ('model_conv1', 'model_bn1'): tensor(0.6577),
  ('model_bn1', 'model_relu'): tensor(0.3459),
  ('model_relu', 'model_maxpool'): tensor(0.4433),
  ('model_maxpool', 'model_layer1_0_conv1'): tensor(0.0293),
  ('model_layer1_0_conv1', 'model_layer1_0_bn1'): tensor(0.0677),
  ('model_layer1_0_bn1', 'model_layer1_0_relu'): tensor(3.2715e-05),
  ('model_layer1_0_relu', 'model_layer1_0_conv2'): tensor(0.0027),
  ('model_layer1_0_conv2', 'model_layer1_0_bn2'): tensor(0.0041),
  ('model_layer1_0_bn2', 'add'): tensor(0.2300),
  ('model_maxpool', 'add'): tensor(1.4507),
  ('add', 'model_layer1_0_relu_1'): tensor(0.4998),
  ('model_layer1_0_relu_1', 'model_layer1_1_conv1'): tensor(0.0578),
  ('model_layer1_1_conv1', 'model_layer1_1_bn1'): tensor(0.0678),
  ('model_layer1_1_bn1', 'model_layer1_1_relu'): tensor(2.4827e-05),
  ('model_layer1_1_relu', 'model_layer1_1_conv2'): tensor(0.0016),
  ('model_layer1_1_conv2', 'model_layer1_1_bn2'): 

In [160]:

inter_std['model_fc'] = torch.ones(1)

inter_std['output'] = torch.ones(1)
grads_sq = {e:g**2  for e,g in zip(edges, grads)}
#sum(grads_sq.values()), grads_sq

In [161]:
# don't process logit edge, so setting its ll to min
grads_sq[('model_fc', 'output')] = max(grads_sq.values())
edges_importance = [grads_sq[e]/max(grads_sq.values()) for e in edges] #[1.0 - edge_ll[e]/max(edge_ll.values()) + EPS for e in edges]
edges_importance

[tensor(0.0004),
 tensor(0.0022),
 tensor(0.0568),
 tensor(0.0728),
 tensor(0.0007),
 tensor(0.0007),
 tensor(5.2817e-06),
 tensor(0.0004),
 tensor(0.0004),
 tensor(0.0103),
 tensor(0.0330),
 tensor(0.0104),
 tensor(0.0012),
 tensor(0.0012),
 tensor(7.4199e-06),
 tensor(0.0005),
 tensor(0.0005),
 tensor(0.0055),
 tensor(0.0032),
 tensor(0.0002),
 tensor(0.0093),
 tensor(0.0057),
 tensor(0.0001),
 tensor(0.0020),
 tensor(0.0020),
 tensor(0.0021),
 tensor(0.0013),
 tensor(0.0136),
 tensor(0.0003),
 tensor(2.5643e-05),
 tensor(0.0010),
 tensor(0.0010),
 tensor(6.2668e-05),
 tensor(0.0003),
 tensor(0.0003),
 tensor(0.0005),
 tensor(5.2240e-05),
 tensor(7.0374e-07),
 tensor(0.0002),
 tensor(0.0001),
 tensor(0.0001),
 tensor(3.1153e-06),
 tensor(3.1154e-06),
 tensor(0.0001),
 tensor(8.6643e-05),
 tensor(0.0005),
 tensor(0.0279),
 tensor(0.0081),
 tensor(0.0006),
 tensor(0.0006),
 tensor(4.2484e-05),
 tensor(0.0007),
 tensor(0.0007),
 tensor(0.0167),
 tensor(0.0117),
 tensor(0.0002),
 tensor(

<!-- ### Conclusion

It seems that the problem is NP-hard. We need to come up with a new approach.
 -->

<!-- # The second attempt

Consider the following heuristic

1. Top sort (v_i, v_j) => i < j

2. for k in {n, ..., 1}

Consider 2 cases:

a) put v_k into the layer of its nearest child + prune some edges

b) put vk into a new layer => 

Consider all subsets of outcoming edges. We instantly identify a layer given a subset. So, we aggregate this layer with the answer of the nearest child to v_k
 -->

## The second attempt + deleting of edges

1. Find all (sample) topological sorts

https://www.geeksforgeeks.org/all-topological-sorts-of-a-directed-acyclic-graph/

2. Apply greedy dynamic programming to find a monotonous solution

3. Postprocess a graph: remove all nodes that are unreacheble from "x".


In [16]:
DG = nx.DiGraph(edges)
all_sorts = list(nx.all_topological_sorts(DG))
len(all_sorts)
rs = np.random.RandomState(42)
rs.shuffle(all_sorts)
all_sorts = all_sorts

/home/legin/.local/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [17]:
len(edges), len(edges_importance)

(78, 78)

In [18]:
# @njit
def dp_for_top_sort(edges, weights, e_importance, top_sort_str, memory=1e10):
    node_ids = {k: i for i, k in enumerate(weights)}
    id_to_node = [node for _, node in enumerate(weights)]
    top_sort = np.array([node_ids[n] for n in top_sort_str])
    assert top_sort[-1] == node_ids['output']
    assert top_sort[0] == node_ids['x']
    assert top_sort.shape[0] == len(node_ids)
    m = np.zeros((len(node_ids), len(node_ids))).astype(np.float32)
    id_to_weight = np.array([weights[n] for n in id_to_node])
    assert len(edges) == len(e_importance)
    
    for (src, dst), w in zip(edges, e_importance):
        src_id, dst_id = node_ids[src], node_ids[dst]
        m[src_id, dst_id] = w
        
    node_to_layers = np.ones((len(node_ids), len(node_ids))).astype(np.int32) * (-100)  # ans for each v ->
    node_to_layers[top_sort[-1], top_sort[-1]] = 0
    dp = [1e9] * len(node_ids)
    dp[top_sort[-1]] = 0
    for i in range(len(node_ids) - 2, -1, -1):
        v = top_sort[i]
        for j in range(i, len(node_ids)):  # the last node of the first layer (starting from v)
            if id_to_weight[top_sort[i: j + 1]].sum() > memory:
                continue
            if j == len(node_ids) - 1:
                dp[v] = 0
                node_to_layers[v, top_sort[i:]] = 0
                continue
            v_j = top_sort[j + 1]
            next_layer_ids = [] if j + 1 >= len(node_ids) else \
            [k for k in range(m.shape[0]) if node_to_layers[v_j, k] == node_to_layers[v_j, v_j]]
            pruned_value = sum([m[top_sort[k], l] for k in range(i, j + 1) for l in next_layer_ids if m[top_sort[k], l] != 0])
            if dp[v_j] + pruned_value <= dp[v]:
                dp[v] = dp[v_j] + pruned_value
                node_to_layers[v] = node_to_layers[v_j]
                node_to_layers[v, top_sort[i:j + 1]] = node_to_layers[v].max() + 1
                
    # prune restricted edges (TODO: also prune unreacheble nodes)
    ans = node_to_layers[node_ids['x']]
    pruned_edges_ids = [(i, j) for i in range(m.shape[0]) for j in range(m.shape[0]) \
                    if m[i, j] != 0 and abs(ans[i] - ans[j]) > 1]
    pruned_edges = [(id_to_node[i], id_to_node[j]) for i, j in pruned_edges_ids]
    pruned_value = sum([m[i, j] for i, j in pruned_edges_ids])
    
    # reach_ids = [set() for _ in range(len(node_ids))]
    # for i in range(len(node_ids) - 1, -1, -1):
    #     v = top_sort[i]
    #     reach_ids[v].add(v)
    #     for k in range(i + 1, m.shape[0]):
    #         v_c = top_sort[k]
    #         if m[v, v_c] != 0 and (v, v_c) not in pruned_edges_ids:
    #             reach_ids[v] |= reach_ids[v_c]
    # conn_g = top_sort[-1] in reach_ids[top_sort[0]]
    conn_g = is_conn(m, node_ids['x'], node_ids['output'])
    if pruned_value == 0:
        assert conn_g
                
    return {'node_to_layer': {id_to_node[i]: ans.max() - l for i, l in enumerate(ans)},
            'pruned_value': pruned_value, 'pruned_edges': pruned_edges,
            'connected_graph': conn_g}



In [19]:
def is_conn(m: np.ndarray, src: int, dst: int):
    reach = [False] * m.shape[0]
    reach[src] = True
    for _ in range(m.shape[0]):
        for i in range(m.shape[0]):
            if reach[i] == False:
                continue
            for j in range(m.shape[0]):
                if m[i, j] != 0:
                    reach[j] = True
    return reach[dst]
    
    

In [ ]:
### find the best solution
import pickle
POW = 1.0

eval_dict = {}
fine_dict = {}

for edge_mem in range(2,6):
    if edge_mem in eval_dict and edge_mem in fine_dict:
        print ('skip', edge_mem)
        continue
 
    model = ResNet18(8)
    model.load_state_dict(torch.load('../model_last.ckpt', map_location='cpu'))
    warp = GraphInterperterWithGamma(model)

    named_dict = dict(model.named_modules())

    edges, weights = module_to_graph(ResNet18())
    # edges, weights
    edges[:10], list(weights.items())[:10]
    
    best_pruned = 1e10
    best_val = None
    for s in tqdm(all_sorts):
        # for mem=8 the computation takes time
        res = dp_for_top_sort(edges, {k: 1 for k in weights}, [np.random.uniform() for _ in edges_importance]
                                  , #[k.item()**POW for k in edges_importance],
                                              s, edge_mem)  
      
        if res['connected_graph'] == True and best_pruned > res['pruned_value']:
            best_pruned = res['pruned_value']
            best_val = res

        if best_pruned == 0:
            print(res)
            break


    model = ResNet18(8)
    model.load_state_dict(torch.load('../model_last.ckpt', map_location='cpu'))

    wrapped = torch.fx.symbolic_trace(model)
    pruned = PrunedModel(wrapped, {k:1.0 if k not in best_val['pruned_edges'] else 0.0 for k in edges }, inter)

    res = test_loop(pruned, test_dl,  "cpu", nc=8)
    if edge_mem not in eval_dict:
        eval_dict[edge_mem] = []
    eval_dict[edge_mem].append(res)

    train_loop(pruned, train_dl, test_dl, 9999999999, 1, 1e-3,  "cpu")
    res = test_loop(pruned, test_dl,  "cpu", nc=8)
    if edge_mem not in fine_dict:
        fine_dict[edge_mem] = []
    fine_dict[edge_mem].append(res)

    import pickle
    with open('molchanov.pckl', 'wb') as out:
        out.write(pickle.dumps([eval_dict, fine_dict]))


Using cache found in /home/legin/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/legin/.cache/torch/hub/pytorch_vision_v0.10.0


  0%|          | 0/9261 [00:00<?, ?it/s]

Using cache found in /home/legin/.cache/torch/hub/pytorch_vision_v0.10.0


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

...

Using cache found in /home/legin/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/legin/.cache/torch/hub/pytorch_vision_v0.10.0


  0%|          | 0/9261 [00:00<?, ?it/s]

Using cache found in /home/legin/.cache/torch/hub/pytorch_vision_v0.10.0


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

In [3]:
import pickle
with open('molchanov.pckl', 'rb') as inp:
    data = pickle.loads(inp.read())
data

[{2: [0.125], 3: [0.125], 4: [0.125], 5: [0.21774999797344208]},
 {2: [0.125], 3: [0.125], 4: [0.6290000081062317], 5: [0.6866250038146973]}]

In [5]:
import pickle
with open('naive_mean.pckl', 'rb') as inp:
    data = pickle.loads(inp.read())
data

[{2: [0.125], 3: [0.125], 4: [0.12600000202655792], 5: [0.2502500116825104]},
 {2: [0.5786250233650208],
  3: [0.5649999976158142],
  4: [0.6498749852180481],
  5: [0.6822500228881836]}]

In [6]:
import pickle
with open('random_mean.pckl', 'rb') as inp:
    data = pickle.loads(inp.read())
data

[{2: [0.12962499260902405, 0.125, 0.125],
  3: [0.125, 0.125, 0.125],
  4: [0.12612499296665192, 0.12612499296665192, 0.125],
  5: [0.12587499618530273, 0.12587499618530273, 0.12562499940395355]},
 {2: [0.5892500281333923, 0.5241249799728394, 0.6132500171661377],
  3: [0.5706250071525574, 0.6041250228881836, 0.5644999742507935],
  4: [0.6047499775886536, 0.6507499814033508, 0.6359999775886536],
  5: [0.6508749723434448, 0.6942499876022339, 0.6966249942779541]}]